In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

Класс линейной интерполяции:

In [2]:
class Interpolator:

    def interpolate(x_list: list, y_list: list, z: float):
        if x_list != sorted(x_list):
            raise ValueError('x_list must be sorted ASC')
        for index, element in enumerate(x_list):
            if z <= element:
                delta = (z - x_list[index - 1]) / (x_list[index] - x_list[index - 1])
                answer = y_list[index - 1] + (y_list[index] - y_list[index - 1]) * delta
                break
        return answer

Пусть у нас есть массив с датами, хотим найти массив с соответствующими своп поинтами и дисконтфакторами рубля.

In [3]:
dates = pd.DataFrame([
    {"dates": '12-04-22'}, 
    {"dates": '03-05-22'},
    {"dates": '19-07-22'},
    {"dates": '17-11-22'},
    {"dates": '21-12-22'},
    {"dates": '04-02-23'},
    {"dates": '24-03-23'},
    {"dates": '16-05-23'},
    {"dates": '18-07-23'},
    {"dates": '05-08-22'},
])


Функция из первой части задания:

In [4]:
def dummy_func(dates):
    return([0.99]*len(dates))

Массив своп поинтов:

In [185]:
swaps = pd.DataFrame([
    {"date": '11-03-22', "swap points":627}, 
    {"date": '12-05-22', "swap points": 1253}, 
    {"date": '21-09-22', "swap points": 1300}, 
    {"date": '29-10-22', "swap points": 2456}, 
    {"date": '01-12-22', "swap points": 1356}, 
    {"date": '11-01-23', "swap points": 3456}, 
    {"date": '15-03-23', "swap points": 5216}, 
    {"date": '16-04-23', "swap points": 6708}, 
    {"date": '10-06-23', "swap points": 8071}, 
    {"date": '26-08-23', "swap points": 9023}])
for a in range(len(swaps)):
    swaps['date'][a] = datetime.datetime.strptime(swaps['date'][a], '%d-%m-%y')
swaps['timestamp'] = swaps['date'].apply(lambda x: x.timestamp())

In [186]:
swaps

,date,swap points,timestamp
0,2022-03-11 00:00:00,627,1.646946e+09
1,2022-05-12 00:00:00,1253,1.652303e+09
2,2022-09-21 00:00:00,1300,1.663708e+09
3,2022-10-29 00:00:00,2456,1.666991e+09
4,2022-12-01 00:00:00,1356,1.669842e+09
5,2023-01-11 00:00:00,3456,1.673384e+09
6,2023-03-15 00:00:00,5216,1.678828e+09
7,2023-04-16 00:00:00,6708,1.681592e+09
8,2023-06-10 00:00:00,8071,1.686344e+09
9,2023-08-26 00:00:00,9023,1.692997e+09


Функция для нахождения своп поинта в неизвестную дату:

In [174]:
def interpolated_swap_point(date):
  date1 = datetime.datetime.strptime(date, '%d-%m-%y')
  return Interpolator.interpolate(list(swaps['timestamp']), list(swaps['swap points']), date1.timestamp())

Считаем массив своп поинтов:

In [178]:
dates['swap_points'] = dates['dates'].apply(interpolated_swap_point)

Нам дан массив дисконт факторов доллара. Найдем ставку доллара:

In [187]:
def years_between(d2, d1 = '5-11-21'): #вспомогательная функция, считает время, прошедшее с сегодняшнего дня
    d1 = datetime.datetime.strptime(d1, "%d-%m-%y")
    d2 = datetime.datetime.strptime(d2, "%d-%m-%y")
    return abs((d2 - d1).days)/365

In [188]:
dates['df_usd'] = dummy_func(dates['dates'])
dates['rate_usd'] = dates['df_usd'].apply(lambda x: -np.log(x)/years_between(dates['dates'][dates['df_usd'] == x].values[0]))

С помощью своп поинтов и ставки доллара находим ставку для рубля:

In [180]:
def get_rate_rub(points, r_usd, spot = 71):
    return((points/(100*spot)+1)*(1+r_usd) - 1)

In [181]:
dates['rate_rub'] = dates.apply(lambda x: get_rate_rub(x.swap_points, x.rate_usd), axis=1)

Считаем дисконт фактор рубля:

In [189]:
dates['df_rub'] = dates['rate_rub'].apply(lambda x: np.exp(-x*years_between(dates['dates'][dates['rate_rub'] == x].values[0])))

Для данного массива дат получили массиы своп поинтов и дисконт фактора рубля:

In [191]:
dates

,dates,swap_points,df_usd,rate_usd,rate_rub,df_rub
0,12-04-22,950.096774,0.99,0.023218,0.160141,0.933027
1,03-05-22,1162.129032,0.99,0.023218,0.190698,0.910719
2,19-07-22,1277.212121,0.99,0.023218,0.207283,0.864692
3,17-11-22,1822.666667,0.99,0.023218,0.285891,0.744315
4,21-12-22,2380.390244,0.99,0.023218,0.366268,0.662041
5,04-02-23,4126.476190,0.99,0.023218,0.617905,0.462107
6,24-03-23,5635.625000,0.99,0.023218,0.835396,0.315520
7,16-05-23,7451.454545,0.99,0.023218,1.097085,0.187461
8,18-07-23,8540.818182,0.99,0.023218,1.254079,0.118812
9,05-08-22,1283.265152,0.99,0.023218,0.208155,0.855825
